In [10]:
# Import necessary libraries
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense, LSTM, SimpleRNN
from datetime import datetime
from pandas_datareader import data as pdr
import warnings

warnings.filterwarnings('ignore')

# Utility function to split sequences
def split_sequence(sequence, n_steps):
    X, y = [], []
    for i in range(len(sequence)):
        end_ix = i + n_steps
        if end_ix > len(sequence) - 1:
            break
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

# Define a function for generating a sequence of future predictions
def sequence_generation(dataset, sc, model, n_steps, steps_future, features):
    high_dataset = dataset.iloc[-(len(dataset) + n_steps):]["High"]
    high_dataset = sc.transform(high_dataset.values.reshape(-1, 1))
    inputs = high_dataset[:n_steps]

    for _ in range(steps_future):
        curr_pred = model.predict(inputs[-n_steps:].reshape(1, n_steps, features), verbose=0)
        inputs = np.append(inputs, curr_pred, axis=0)

    return sc.inverse_transform(inputs[n_steps:])

# Generic training function for RNN or LSTM
def train_model(model_type, X_train, y_train, n_steps, features, sc, test_set, dataset, epochs=10, batch_size=32, verbose=1, steps_in_future=25, save_model_path=None):
    model = Sequential()
    if model_type == "RNN":
        model.add(SimpleRNN(units=125, input_shape=(n_steps, features)))
    elif model_type == "LSTM":
        model.add(LSTM(units=125, input_shape=(n_steps, features)))
    else:
        raise ValueError("Invalid model type. Choose 'RNN' or 'LSTM'.")
        
    model.add(Dense(units=1))
    model.compile(optimizer="RMSprop", loss="mse")
    
    # Train the model
    model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=verbose)
    
    # Prepare test data
    inputs = sc.transform(test_set.reshape(-1, 1))
    X_test, y_test = split_sequence(inputs, n_steps)
    X_test = X_test.reshape(-1, n_steps, features)

    # Predict and evaluate
    predicted_stock_price = model.predict(X_test, verbose=0)
    predicted_stock_price = sc.inverse_transform(predicted_stock_price)
    rmse = np.sqrt(mean_squared_error(sc.inverse_transform(y_test.reshape(-1, 1)), predicted_stock_price))
    print(f"The root mean squared error is {rmse:.2f}.")
    
    # Generate future predictions
    results = sequence_generation(dataset, sc, model, n_steps, steps_in_future, features)
    print("Generated sequence of future predictions:", results.flatten())
    
    # Save the model
    if save_model_path:
        model.save(save_model_path)
        print(f"Model saved successfully at {save_model_path}.")
    
    return model

# Load historical stock price data for AAPL
yf.pdr_override()
dataset = pdr.get_data_yahoo('AAPL', start='2012-01-01', end=datetime.now())
print("Data Loaded")

# Split dataset into training and test sets
train_end = datetime(2020, 1, 1)
training_set = dataset[dataset.index < train_end]["High"].values
test_set = dataset[dataset.index >= train_end]["High"].values

# Scale dataset values
sc = MinMaxScaler(feature_range=(0, 1))
training_set_scaled = sc.fit_transform(training_set.reshape(-1, 1))

# Create overlapping window batches
n_steps = 50
features = 1
X_train, y_train = split_sequence(training_set_scaled, n_steps)

# Reshape X_train for compatibility
X_train = X_train.reshape(X_train.shape[0], n_steps, features)

# Train RNN model
model_rnn = train_model("RNN", X_train, y_train, n_steps, features, sc, test_set, dataset, epochs=10, batch_size=32, steps_in_future=25, save_model_path="output/model_rnn.h5")

# Train LSTM model
model_lstm = train_model("LSTM", X_train, y_train, n_steps, features, sc, test_set, dataset, epochs=10, batch_size=32, steps_in_future=25, save_model_path="output/model_lstm.h5")


yfinance: pandas_datareader support is deprecated & semi-broken so will be removed in a future verison. Just use yfinance.


[*********************100%%**********************]  1 of 1 completed


Data Loaded
Epoch 1/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.1039
Epoch 2/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0026
Epoch 3/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0019
Epoch 4/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 8.8233e-04
Epoch 5/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 4.2676e-04
Epoch 6/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 2.9091e-04
Epoch 7/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 2.2181e-04
Epoch 8/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 2.6120e-04
Epoch 9/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 2.8716e-04
Epoch 10/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 2.2924e-04
The root mean squared error is 13.17.


Generated sequence of future predictions: [21.04286473 21.42901189 21.56553064 21.90959403 22.15586644 22.32975219
 22.55957813 22.88607169 22.95050824 23.19456743 23.34252907 23.60179486
 23.74222291 23.99980055 24.19401471 24.44751713 24.67536195 24.9512145
 25.19688065 25.48828926 25.74178462 25.98915746 26.27167613 26.53618816
 26.76120204]
Model saved successfully at output/model_rnn.h5.
Epoch 1/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - loss: 0.0180
Epoch 2/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 0.0014
Epoch 3/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 9.2404e-04
Epoch 4/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 5.9626e-04
Epoch 5/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 6.1198e-04
Epoch 6/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 4.1995e-04
Epoch 7/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - loss: 3.8861e-04
Epoch 8/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: 3.4433e-04
Epoch 9/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step -

Generated sequence of future predictions: [20.32620572 20.46905349 20.60137966 20.7253317  20.84303767 20.95604276
 21.06547696 21.17216141 21.27670588 21.37955912 21.48106572 21.58148116
 21.68100177 21.77978288 21.87793568 21.97555812 22.07273139 22.1694996
 22.26591395 22.36201326 22.45782489 22.55336824 22.64866363 22.74373575
 22.83858285]
Model saved successfully at output/model_lstm.h5.


In [13]:
model_lstm.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 125)            │        63,500 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           126 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 127,254 (497.09 KB)

 Trainable params: 63,626 (248.54 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 63,628 (248.55 KB)